In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import unicodedata as ud
import math
import scipy.integrate as integrate
from scipy.integrate import solve_ivp, odeint, complex_ode
from qutip import *
from mpl_toolkits.mplot3d import Axes3D
from scipy import optimize

In [ ]:
tlist = np.linspace(0, 200, 201)
taulist = np.linspace(10,200,191) # kist of integration times tau65

In [ ]:
# parameters. Frequency in GHz, time in ns

w_r = 4.754 * 2 * np.pi  # readout resonator frequency
w_d = w_r
w_q = 6.316 * 2 * np.pi  # qubit frequency
w_f = 4.752 * 2 * np.pi
Delta = w_q - w_r # qubit resonator detuning
delta = 340e-3 * 2 * np.pi # transmon anharmonicity
# don't mix these two!
g = 208e-3 * 2 * np.pi 
G = 25e-3 *2 * np.pi
T_1 = 7.6e3
n_crit = Delta**2/(4*g**2)
n_d = 2.5
k_f = 64e-3 * 2 * np.pi

chi = g**2/Delta - g**2/(Delta - delta)
k_eff_q = 4*G**2/k_f/(1 + (2*(w_q - w_f)/k_f)**2)
k_eff_d = 4*G**2 / k_f/ (1 + (2*(w_r - w_f)/k_f)**2)
E_d = np.sqrt(n_d*(k_eff_d**2/4 + chi**2))
Gamma = g**2 * k_eff_q / Delta**2 

In [ ]:
# cavity operators 
N=20
a = tensor(destroy(N), qeye(2)) # annihilation 
nc = a.dag() * a
xr1 = (a + a.dag())
xr2 = 1/(1j)*(a - a.dag())

# qubit operators 
sz = tensor(qeye(N), sigmaz()) # sigma_z = |e><e|-|g><g|
sp = tensor(qeye(N), sigmap()) # sigma_+
sm = tensor(qeye(N), sigmam()) # sigma_-
sx = sp + sm
nq = sp * sm

In [ ]:
psi0_g = tensor(basis(N,0), basis(2, 1))
psi0_e = tensor(basis(N,0), basis(2, 0))

In [ ]:
H_JC = w_q * sz + (w_r)* nc + chi*nc*sz
H_JC_int = (w_q- w_d)/2*sz + (w_r- w_d)*nc + chi*nc*sz 
H_int = H_JC_int + E_d * (a + a.dag())

In [ ]:
res_g_qss = mesolve(H_int, psi0_g, tlist, [np.sqrt(k_eff_d) * a, np.sqrt(1/T_1) * sm, np.sqrt(Gamma) * sm], [xr1]).expect[0]
res_e_qss = mesolve(H_int, psi0_e, tlist, [np.sqrt(k_eff_d) * a, np.sqrt(1/T_1) * sm, np.sqrt(Gamma) * sm], [xr1]).expect[0]

## Fig 1(c): cavity quadrature trajectories

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 4))

ax.plot(tlist, res_e_qss/2, label = 'Excited', c='r')
ax.plot(tlist, res_g_qss/2, label = 'Ground', c='g')

ax.legend()
ax.set_xlabel("Time (ns)")
ax.set_ylabel("Q(t)")
ax.grid(True)

## Tuning $\chi$ and $\chi/\kappa$

In [ ]:
def find_G(k_r):
    def func(G_r):
        return 4*G_r**2 / k_f/ (1 + (2*(w_r - w_f)/k_f)**2) - k_r # k is k_eff_d for given ratio
    s = optimize.root_scalar(func, bracket=[2*np.pi*1e-3, 2*np.pi*50e-3], method='brentq').root
    return s

### $\chi$=-7.7 MHz, $\chi/\kappa$=0.5

In [ ]:
r = 0.5
g_7_05 = 208e-3 * 2 * np.pi
chi_7_05 = -2*np.pi*7.7e-3
k_eff_d_7_05 = np.abs(chi_7_05/r)
E_d_7_05 = np.sqrt(n_d*(k_eff_d_7_05**2/4 + chi_7_05**2))
G_7_05 = find_G(k_eff_d_7_05)
k_eff_7_05 = 4*G_7_05**2/k_f/(1 + (2*(w_q - w_f)/k_f)**2)
Gamma_7_05 = g_7_05**2 * k_eff_7_05 / Delta**2 

H_JC_int = (w_q- w_d)/2*sz + (w_r- w_d)*nc + chi_7_05*nc*sz
H_int = H_JC_int + E_d_7_05 * (a + a.dag())

Qg_7_05 = mesolve(H_int, psi0_g, tlist, [np.sqrt(k_eff_d_7_05) * a, np.sqrt(1/T_1) * sm, np.sqrt(Gamma_7_05) * sm], [xr1, xr2]).expect[0]
Qe_7_05 = mesolve(H_int, psi0_e, tlist, [np.sqrt(k_eff_d_7_05) * a, np.sqrt(1/T_1) * sm, np.sqrt(Gamma_7_05) * sm], [xr1, xr2]).expect[0]

### $\chi$=-4 MHz, $\chi/\kappa$=0.5

In [ ]:
r = 0.5
g_4_05 = 208/np.sqrt(7.7/4)*1e-3 * 2 * np.pi
chi_4_05 = -2*np.pi*4e-3
k_eff_d_4_05 = np.abs(chi_4_05/r)
E_d_4_05 = np.sqrt(n_d*(k_eff_d_4_05**2/4 + chi_4_05**2))
G_4_05 = find_G(k_eff_d_4_05)
k_eff_4_05 = 4*G_4_05**2/k_f/(1 + (2*(w_q - w_f)/k_f)**2)
Gamma_4_05 = g_4_05**2 * k_eff_4_05 / Delta**2 

H_JC_int = (w_q- w_d)/2*sz + (w_r- w_d)*nc + chi_4_05*nc*sz
H_int = H_JC_int + E_d_4_05 * (a + a.dag())

Qg_4_05 = mesolve(H_int, psi0_g, tlist, [np.sqrt(k_eff_d_4_05) * a, np.sqrt(1/T_1) * sm, np.sqrt(Gamma_4_05) * sm], [xr1, xr2]).expect[0]
Qe_4_05 = mesolve(H_int, psi0_e, tlist, [np.sqrt(k_eff_d_4_05) * a, np.sqrt(1/T_1) * sm, np.sqrt(Gamma_4_05) * sm], [xr1, xr2]).expect[0]

### $\chi$=-2 MHz, $\chi/\kappa$=0.5

In [ ]:
r = 0.5
g_2_05 = 208/np.sqrt(7.7/2)*1e-3 * 2 * np.pi
chi_2_05 = -2*np.pi*2e-3
k_eff_d_2_05 = np.abs(chi_2_05/r)
E_d_2_05 = np.sqrt(n_d*(k_eff_d_2_05**2/4 + chi_2_05**2))
G_2_05 = find_G(k_eff_d_2_05)
k_eff_2_05 = 4*G_2_05**2/k_f/(1 + (2*(w_q - w_f)/k_f)**2)
Gamma_2_05 = g_2_05**2 * k_eff_2_05 / Delta**2 

H_JC_int = (w_q- w_d)/2*sz + (w_r- w_d)*nc + chi_2_05*nc*sz
H_int = H_JC_int + E_d_2_05 * (a + a.dag())

Qg_2_05 = mesolve(H_int, psi0_g, tlist, [np.sqrt(k_eff_d_2_05) * a, np.sqrt(1/T_1) * sm, np.sqrt(Gamma_2_05) * sm], [xr1, xr2]).expect[0]
Qe_2_05 = mesolve(H_int, psi0_e, tlist, [np.sqrt(k_eff_d_2_05) * a, np.sqrt(1/T_1) * sm, np.sqrt(Gamma_2_05) * sm], [xr1, xr2]).expect[0]

## Fig 4(b): S(t) for different parameters

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(4, 4))

ax.plot(tlist, np.sqrt(k_f)*(res_e_qss/2-res_g_qss/2), label='7.7 0.2',c='b', linestyle='--')
ax.plot(tlist, np.sqrt(k_f)*(Qe_2_05/2-Qg_2_05/2),label='2 0.5', c='r')
ax.plot(tlist, np.sqrt(k_f)*(Qe_4_05/2-Qg_4_05/2),label='4 0.5', c='g')
ax.plot(tlist, np.sqrt(k_f)*(Qe_7_05/2-Qg_7_05/2),label='7.7 0.5', c='b')
ax.set_xlabel("Time (ns)")
ax.set_ylabel("S(t)")
ax.legend()
ax.grid(True)

In [ ]:
def iir(Qe, Qg, tau): # integrated information rate
    i = list(tlist).index(tau)
    yvalues = np.sqrt(k_f)*(Qe/2-Qg/2)
    return integrate.trapezoid(yvalues[0:i+1], tlist[0:i+1]) / np.sqrt(tau)

## Fig 4(c): s($\tau$) for different parameters

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 4))

plt.yscale("log")
plt.xscale("log")
ax.plot(taulist, list(map(lambda tau: iir(res_e_qss, res_g_qss, tau), taulist)), label='7.7 0.2', c='b', linestyle='--')
ax.plot(taulist, list(map(lambda tau: iir(Qe_2_05, Qg_2_05, tau), taulist)), label='2 0.5', c='r')
ax.plot(taulist, list(map(lambda tau: iir(Qe_4_05, Qg_4_05, tau), taulist)), label='4 0.5', c='g')
ax.plot(taulist, list(map(lambda tau: iir(Qe_7_05, Qg_7_05, tau), taulist)), label='7.7 0.5', c='b')
ax.set_xlabel("Integration time (ns)")
ax.legend()
ax.grid(True)
ax.axvline(64, c='grey',  linestyle = 'dotted')
ax.set_ylabel(r'  S($\tau$)');

In [ ]:
def simulate_readout(timesteps, chi, k_eff_d):
    
    N = 20
    
    sm = tensor(sigmam(), qeye(N)) # qubit
    sz = tensor(-sigmaz(), qeye(N)) # qubit
    a = tensor(qeye(2), destroy(N)) # readout resonator
    Qr = (a + a.dag()) / 2

    c_ops = [np.sqrt(k_eff_d) * a, np.sqrt(1/T_1) * sm, np.sqrt(Gamma) * sm]

    psi0_g = tensor(basis(2,0), basis(N, 0))
    psi0_e = tensor(basis(2,1), basis(N, 0))
    
    E_d = np.sqrt(n_d*(k_eff_d**2/4 + chi**2))
    
    H_JC_int = (w_q - w_d) * sz + (w_r - w_d) * a.dag()*a + chi*a.dag()*a * sz
    H_drive = E_d * (a + a.dag())
    H_int = H_JC_int + H_drive

    result_g = mesolve(H_int, psi0_g, timesteps, c_ops)
    result_e = mesolve(H_int, psi0_e, timesteps, c_ops)

    Qr_0 = expect(Qr, result_g.states)
    Qr_1 = expect(Qr, result_e.states)
    
    return Qr_0, Qr_1

def integrated_info_rate(timesteps, Qr_0, Qr_1, k_eff):
    t_min = np.min(timesteps)
    t_max = np.max(timesteps)
    tau = t_max - t_min
    return 1 / np.sqrt(tau) * np.sum(np.abs(np.sqrt(k_eff) * np.real(Qr_0 - Qr_1)))

timesteps = np.linspace(0,200,1000)

chis = [-7.9] # MHz
chis = np.array(chis) * 2 * np.pi * 1e-3
ratios = np.linspace(0.05, 0.7, 100)
readout_timesteps = [timesteps[:i] for i in range(60, len(timesteps), 20)]
readout_durations = [timesteps[i] for i in range(60, len(timesteps), 20)]

results = []
for chi in chis:
    temp = []
    for ratio in ratios:
        k_eff = np.abs(chi / ratio)
        Qr_0, Qr_1 = simulate_readout(timesteps, chi, k_eff)
        info_rates = []
        for i, readout_timestep in enumerate(readout_timesteps):
            iir = integrated_info_rate(readout_timestep, Qr_0[:(60 + 20*(i+1))], Qr_1[:(60 + 20*(i+1))], k_eff)
            info_rates.append(iir)
        
        temp.append(info_rates)
    results.append(temp)
    

## Fig 4(a): optimal ratio vs integration time

In [ ]:
r = np.array(results)

chi1 = r[0,:,:]
max_ratios = []
for i in range(chi1.shape[1]):
    # print(np.argmax(chi1[:,i]))
    max_ratio = ratios[np.argmax(chi1[:,i])]
    max_ratios.append(max_ratio)
    
fig, ax = plt.subplots()
ax.plot(np.array(readout_durations) * -chis[0], max_ratios, 'o-')
ax.set_xlabel("Integration time (ns)")
ax.set_ylabel("Optimal ratio")